In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')
plt.rc('patch', force_edgecolor=True,edgecolor='black')
plt.rc('hist', bins='auto')

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten, Dropout

from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

In [96]:
train = pd.read_csv('wrangled_data/training_post_wrangle_small.csv')

In [105]:
X = train.set_index(['ID', 'date_block_num'])['item_cnt_month'].values.reshape((214200, 34))

In [106]:
X_train = X[:, :-1]
y_train = X[:, -1]

X_test = X[:, 1:]

In [110]:
print(X_train.shape, y_train.shape, X_test.shape)

(214200, 33) (214200,) (214200, 33)


https://machinelearningmastery.com/how-to-use-the-timeseriesgenerator-for-time-series-forecasting-in-keras/

In [109]:
generator = tf.keras.preprocessing.sequence.TimeseriesGenerator(X_train, y_train, length=1)

In [115]:
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(1, 33)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [116]:
model.fit_generator(generator, steps_per_epoch=1, epochs=5, verbose=0)

In [120]:
yhat = model.predict(X_test.reshape((214200, 1, 33)), verbose=0)

In [123]:
yhat = yhat.clip(0,20)

In [130]:
submission = pd.DataFrame(yhat).reset_index().rename(columns={'index':'ID', 0:'item_cnt_month'})

In [131]:
submission.to_csv('submission_files/timeseriesgenerator.csv',index = False)

In [132]:
submission.head(3)

,ID,item_cnt_month
0,0,0.000000
1,1,0.008074
2,2,0.092263


In [133]:
submission.shape

(214200, 2)